# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [53]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [54]:
csv = "../output_data/cities.csv"
city_data = pd.read_csv(csv)

city_data_df = pd.DataFrame(city_data)
city_data_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,limon,10.0000,-83.0333,75.22,82,100,5.95,CR,1610163351
1,arraial do cabo,-22.9661,-42.0278,76.62,89,85,3.36,BR,1610163351
2,poum,-20.2333,164.0167,83.61,76,90,10.13,NC,1610163352
3,port alfred,-33.5906,26.8910,71.01,95,8,4.74,ZA,1610163338
4,begoro,6.3871,-0.3774,70.95,96,60,4.09,GH,1610163352
...,...,...,...,...,...,...,...,...,...
563,guatire,10.4762,-66.5427,77.00,69,90,6.91,VE,1610163487
564,kiunga,-6.1219,141.2906,92.03,49,100,3.31,PG,1610163487
565,nemuro,43.3236,145.5750,19.40,58,20,11.50,JP,1610163487
566,sao joao,-8.8756,-36.3667,68.36,90,68,8.95,BR,1610163488


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [61]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = city_data_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
humidity = city_data_df["Humidity"]



In [62]:
#Plot Heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius = 2)

#Add layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [57]:
# DataFrame to find your ideal weather condition.
ideal_conditions_df = city_data_df

#Ideal conditions
ideal_conditions_df= ideal_conditions_df.loc[(ideal_conditions_df["Max Temp"] < 80) & (ideal_conditions_df["Max Temp"] > 70)]
ideal_conditions_df = ideal_conditions_df.loc[ideal_conditions_df["Wind Speed"] < 15]
ideal_conditions_df = ideal_conditions_df.loc[ideal_conditions_df["Cloudiness"] == 0]

# Drop any rows with null values
ideal_conditions_df = ideal_conditions_df.dropna()

ideal_conditions_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
83,saint-pierre,-21.3393,55.4781,78.80,57,0,3.44,RE,1610163372
94,cidreira,-30.1811,-50.2056,74.21,61,0,10.80,BR,1610163374
141,kidal,18.4411,1.4078,72.77,19,0,9.95,ML,1610163385
159,goundam,16.4145,-3.6708,72.18,19,0,7.54,ML,1610163389
165,upala,10.8979,-85.0145,78.80,69,0,9.22,CR,1610163390
177,allapalli,19.4167,80.0667,79.30,43,0,2.17,IN,1610163393
263,tahoua,14.8888,5.2692,71.60,24,0,6.91,NE,1610163413
281,rio grande,-32.0350,-52.0986,71.04,87,0,12.48,BR,1610163407
287,natitingou,10.3042,1.3796,71.60,46,0,2.30,BJ,1610163420
296,ipixuna,-1.7625,-48.8043,77.00,88,0,4.61,BR,1610163423


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [75]:
# Create a hotel_df
hotel_df = ideal_conditions_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
83,saint-pierre,RE,-21.3393,55.4781,
94,cidreira,BR,-30.1811,-50.2056,
141,kidal,ML,18.4411,1.4078,
159,goundam,ML,16.4145,-3.6708,
165,upala,CR,10.8979,-85.0145,
177,allapalli,IN,19.4167,80.0667,
263,tahoua,NE,14.8888,5.2692,
281,rio grande,BR,-32.0350,-52.0986,
287,natitingou,BJ,10.3042,1.3796,
296,ipixuna,BR,-1.7625,-48.8043,


In [80]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# params dictionary to update each iteration
params = {"type" : "lodgin",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f" Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
 
    print(response)

    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

break    

print("-------End of Search-------")


 Results for Index 83: saint-pierre.
{'error_message': 'You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
Missing field/result... skipping.
------------
 Results for Index 94: cidreira.
{'error_message': 'You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
Missing field/result... skipping.
------------
 Results for Index 141: kidal.
{'error_message': 'You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
Miss

SyntaxError: 'break' outside loop (<ipython-input-80-e299e6969417>, line 41)

In [48]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
83,saint-pierre,RE,-21.3393,55.4781,
94,cidreira,BR,-30.1811,-50.2056,
141,kidal,ML,18.4411,1.4078,
159,goundam,ML,16.4145,-3.6708,
165,upala,CR,10.8979,-85.0145,
177,allapalli,IN,19.4167,80.0667,
263,tahoua,NE,14.8888,5.2692,
281,rio grande,BR,-32.0350,-52.0986,
287,natitingou,BJ,10.3042,1.3796,
296,ipixuna,BR,-1.7625,-48.8043,


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [52]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))